In [0]:
# GOLD LAYER - KPIs Y AGREGACIONES PARA ANALÍTICA FINAL
# ========================================================

from pyspark.sql.functions import col, round, sum, countDistinct, avg, to_date

# 1. Cargar desde Silver Layer
df_silver = spark.read.format("delta").load("/Volumes/workspace/default/retail_lakehouse/silver")

# 2. Calcular columna TotalVenta (por si no está)
df_silver = df_silver.withColumn("TotalVenta", round(col("Quantity") * col("UnitPrice"), 2))

# 3. KPI: Total de ventas por país
df_kpi_pais = (
    df_silver
    .groupBy("Country")
    .agg(sum("TotalVenta").alias("VentasTotales"))
    .orderBy("VentasTotales", ascending=False)
)
df_kpi_pais.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/gold/kpi_ventas_pais")

# 4. KPI: Productos más vendidos
df_kpi_productos = (
    df_silver
    .groupBy("Description")
    .agg(sum("TotalVenta").alias("TotalVendido"))
    .orderBy("TotalVendido", ascending=False)
)
df_kpi_productos.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/gold/kpi_productos_top")

# 5. KPI: Ventas diarias
df_kpi_dia = (
    df_silver
    .withColumn("Fecha", to_date("InvoiceDate"))
    .groupBy("Fecha")
    .agg(sum("TotalVenta").alias("VentasTotales"))
    .orderBy("Fecha")
)
df_kpi_dia.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/gold/kpi_ventas_dia")

# 6. KPI: Clientes con mayor gasto
df_kpi_clientes = (
    df_silver
    .groupBy("CustomerID")
    .agg(sum("TotalVenta").alias("TotalGastado"))
    .orderBy("TotalGastado", ascending=False)
)
df_kpi_clientes.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/gold/kpi_clientes_top")

# 7. KPI: Ticket promedio por transacción
df_kpi_ticket = (
    df_silver
    .groupBy("InvoiceNo")
    .agg(sum("TotalVenta").alias("MontoFactura"))
    .agg(round(avg("MontoFactura"), 2).alias("TicketPromedio"))
)
df_kpi_ticket.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/gold/kpi_ticket_promedio")

# 8. (Opcional) Crear vistas temporales

# df_kpi_pais.createOrReplaceTempView("gold_ventas_por_pais")
# df_kpi_productos.createOrReplaceTempView("gold_productos_top")
# df_kpi_dia.createOrReplaceTempView("gold_ventas_diarias")
# df_kpi_clientes.createOrReplaceTempView("gold_clientes_top")
# df_kpi_ticket.createOrReplaceTempView("gold_ticket_promedio")

# ✅ Fin de la capa Gold